In [10]:
from flask import Flask, request
# from flask_sqlalchemy import SQLAlchemy
from flask_cors import CORS
import pandas as pd
from IPython.display import display
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import snscrape.modules.twitter as sntwitter
import re
import demoji
import datetime
import math

In [23]:
userInput = pd.read_csv("E:/JDE/JDE-TweetsAnalysis/backend/result.csv", index_col=0)
userInput

,id,name,cashtag,qFilter,qFilterLinks,qFilterReplies,lang,qFilterVerified,qLocation,qStartTime,qEndTime,qWithinTime,qMinLike,qMinRetweets,qMinReplies,sa_rmEmoji,anaURL
0,041f7e8d-dc04-465a-882c-8ff216e41bf6,honda,cbr,NaN,True,True,en,False,NaN,NaN,NaN,NaN,0,0,0,True,False
1,4aac49d4-c14d-4adf-860d-2f085f8fec44,Suzuki,NaN,"cars, suvs,marine",False,False,zh,True,us,NaN,NaN,NaN,5,9,0,False,True


In [77]:
def reFilterSpace(text):
    text = str(text)
    while text[0] == ' ':
        text = text[1:]
    while text[-1] == ' ':
        text = text[:-1]
        reFilterSpace(text)
    return text

def sentiment_scores(sentence):
    sid_obj = SentimentIntensityAnalyzer()
    sentiment_dict = sid_obj.polarity_scores(sentence)
    return sentiment_dict

def rmURL(content, condition=True):
    return (re.sub(r'http\S+', '', content) if condition else content)

def findURL(content):
    return re.findall(r'http\S+', content)

def rmEmoji(content):
    return demoji.replace(content, repl='')

def grapTweets(name, cashTag, qFilter, qFilterLinks, qFilterReplies, lang, qFilterVerified, qLocation, qStartTime, qEndTime, qWithinTime, qMinLike, qMinRetweets, qMinReplies, searchTime, id, samples):
    # init #
    orCashTag = ''
    qFilterText = ''
    addfilter = ''
    cashTag = str(cashTag)
    qFilter = str(qFilter)
    qWithinTime = reFilterSpace(str(qWithinTime))

    ### Transform User input to Query ###
    if (cashTag == '') or (cashTag!='nan'):
        for text in cashTag.split(','):
            orCashTag += f' OR "{reFilterSpace(text)}"'
    if (qFilter == '') or (qFilter!='nan'):
        for text in qFilter.split(','):
            qFilterText += f' -"{reFilterSpace(text)}"'
    if qFilterLinks:
        addfilter += ' -filter:links'
    if qFilterReplies:
        addfilter += ' -filter:replies'
    if qFilterVerified:
        addfilter += ' filter:verified'
    if type(qLocation)==str: # NaN's type is float
        addfilter += f' near:"{reFilterSpace(qLocation)}"'
    if (qWithinTime == '') or (qWithinTime=='nan'):
        if str(qStartTime) != 'nan':
            qStartTime = str(qStartTime).split('.')[0]
            addfilter += f' since_time:{qStartTime}'
        if str(qEndTime) != 'nan':
            qStartTime = str(qEndTime).split('.')[0]
            addfilter += f' until_time:{qEndTime}'
    else:
        addfilter += f' within_time:{qWithinTime}'

    ### Grapping ###
    queryText = f'(({name}){orCashTag}){qFilterText}{addfilter} lang:{lang} min_retweets:{qMinRetweets} min_faves:{qMinLike} min_replies:{qMinReplies}'
    scraper = sntwitter.TwitterSearchScraper(queryText)
    print(queryText)
    tweets = []
    for i, tweet in enumerate(scraper.get_items()):
        data = [
            tweet.date,
            tweet.id,
            tweet.content,
            tweet.user.username,
            tweet.likeCount,
            tweet.retweetCount,
            tweet.url,
            searchTime,
            id,
            name
        ]
        tweets.append(data)
        ### Grap how many Data ###
        if i==samples:
            break
    return pd.DataFrame(tweets, columns=['date', 'id', 'content', 'username', 'likes', 'retweets', 'url', 'from_query_time', 'from_query_id', 'from_query_name'])

def returnGrap(row, input, samples=20):
    # print(
    return grapTweets(
        input.loc[row, 'name'],
        input.loc[row, 'cashtag'],
        input.loc[row, 'qFilter'],
        input.loc[row, 'qFilterLinks'],
        input.loc[row, 'qFilterReplies'],
        input.loc[row, 'lang'],
        input.loc[row, 'qFilterVerified'],
        input.loc[row, 'qLocation'],
        input.loc[row, 'qStartTime'],
        input.loc[row, 'qEndTime'],
        input.loc[row, 'qWithinTime'],
        input.loc[row, 'qMinLike'],
        input.loc[row, 'qMinRetweets'],
        input.loc[row, 'qMinReplies'],
        input.loc[row, 'searchTime'],
        input.loc[row, 'id'],
        samples
    )

In [74]:
now = datetime.datetime.now().timestamp()
userInput['searchTime'] = now

In [99]:
tweets = pd.DataFrame()
for i in range(0, len(userInput)):
    tweets = pd.concat([tweets, returnGrap(i, userInput, 2)])
tweets.drop(columns=['from_query_time', 'from_query_id', 'id'], inplace=False)

((honda) OR "cbr") -filter:links -filter:replies within_time:180d lang:en min_retweets:0 min_faves:0 min_replies:0


C:\Users\Killicit\AppData\Local\Temp\ipykernel_12196\2619861878.py:67: FutureWarning: content is deprecated, use rawContent instead
  tweet.content,


((Suzuki)) -"cars" -"suvs" -"marine" filter:verified near:"us" since_time:1609430400 lang:en min_retweets:9 min_faves:5 min_replies:0


C:\Users\Killicit\AppData\Local\Temp\ipykernel_12196\2619861878.py:67: FutureWarning: content is deprecated, use rawContent instead
  tweet.content,


,date,content,username,likes,retweets,url,from_query_name
0,2023-02-02 22:42:28+00:00,Honestly I could beat an ROTC guy. What’s he g...,2002HondaAccord,0,0,https://twitter.com/2002HondaAccord/status/162...,honda
1,2023-02-02 22:40:01+00:00,CBR Commercial,CBeastradio,0,0,https://twitter.com/CBeastradio/status/1621277...,honda
2,2023-02-02 22:39:12+00:00,"On Nov 12, Kamehameha beat Konawaena, 47-36. a...",PupulePaul,1,0,https://twitter.com/PupulePaul/status/16212770...,honda
0,2023-02-02 13:01:37+00:00,Should we cool the Earth with aircraft sprayin...,DavidSuzukiFDN,18,11,https://twitter.com/DavidSuzukiFDN/status/1621...,Suzuki
1,2023-02-01 01:40:27+00:00,Rafael Harvey-Pinard has taken Rem Pitlick's s...,EricEngels,336,13,https://twitter.com/EricEngels/status/16205979...,Suzuki
2,2023-01-31 15:24:30+00:00,"The mystery around Sean Monahan's return, Nick...",Sportsnet,66,13,https://twitter.com/Sportsnet/status/162044290...,Suzuki


In [98]:
displayResult = [tweets[tweets['from_query_id']==i] for i in tweets['from_query_id'].unique()]
type(displayResult)

list

In [97]:
type(displayResult[0].to_html())

str